<a href="https://colab.research.google.com/github/MarcosACVieira/0_GERAL/blob/main/Sum%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ideia geral

Criar uma ou mais redes neurais e entender como ela aprende ao longo do processo de interações. Para isso a rede pode ter um treinamento inicial e uma sequência de 'predições' e 'ações' e aprendizados' sucessivos.

-Predição: levata-se o que a rede provavelmente faria em uma situação.

-Ação: baseado no que a rede provavelmente faria, ela toma uma ação.

-Aprendizado: a ação produz um resultado que gera um apredizado novo para a rede.

O novo aprendizado muda o estado da RN em si, o que faz com que a próxima predição tenha resultados diferentes, mais ou menos deslocados da predição anterior


# Caso 1: Pé na porta.

Objetivo do caso: Observar como aprendizados incrementais deslocam a posição de uma rede

Treinamento inicial: a rede é treinada com um entendimento sobre se ela deveria ou não aceitar tarefas de aversividades maiores ou menores.

Predição: uma RN indica se deveria (ou não) acatar à solicitação de uma tarefa com aversifidade mais (ou menos) elevada.

Ação: ela executa (ou não) a tarefa acatando a solicitação.

Aprendizado: ao acatar (ou não) a solicitação há um aprendizado incremental de que essa solicitação deve (ou não) se acatada

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
import matplotlib

# Treinamento inicial
## aversividade
input_firts = [[0.0],
               [0.5],
               [1.0]]

## chance de aceitar
output_firts = [[0.8],
                [0.5],
                [0.2]]

l_rate = 0.7
model = keras.Sequential()
model.add(layers.Dense(10, activation="sigmoid"))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=l_rate, momentum = 0.0), loss = tf.keras.losses.MeanSquaredError())


## Treina a rede nessas respostas
model.fit(input_firts, output_firts, epochs=1000, verbose=0)

## Salva as respotas originais da rede para comparações
original_weights = model.get_weights()


steps = 20
rounds = 100
verbose = 0
rounds_acc = []
count = 0
for round in range(rounds) :
    odds_before_acc = []
    odds_original_acc = []
    odds_after_acc = []
    accept_acc = []
    averse_acc = []
    model.set_weights(original_weights)
    if verbose == 1 : print("-----------------")
    if verbose == 1 : print("### Rodada:",round)
    if verbose == 1 : print("-----------------")

    for i in range(steps+1) :
        count = count+1
        if verbose == 1 : print("Fase: ",i)
        if verbose == 1 : print("Porcentagem decorrida: ",count/(steps*rounds))
        if verbose != 1 : print("\r{:.0%}".format(count/(steps*rounds+rounds)),end="")

        # input no momento
        if verbose == 1 : print("Aversividade: ",i/steps)
        input_training =[[i/steps]]
        input_training = tf.convert_to_tensor(input_training)
        averse_acc.append(i/steps)

        # resposta original que a rede daria, para fins de comparação
        curr_weights = model.get_weights()
        model.set_weights(original_weights)
        odds_original = model.predict(input_training, verbose=0).squeeze()
        if verbose == 1 : print("Originalmente a chance era: {:.0%}".format(odds_original) )
    
        # volta a rede para o estado atual
        odds_original_acc.append(odds_original.tolist())
        model.set_weights(curr_weights)

        # resposta que a rede dá nesse momento, antes de aceitar ou não a solicitação
        odds_before = model.predict(input_training, verbose=0).squeeze()
        if verbose == 1 : print("Chance de aceitar: {:.0%}".format(odds_before) )
        odds_before_acc.append(odds_before.tolist())

        # sendo a resposta atual uma probabilidade, a rede vai aceitar ou não
        dice = random.random()
        if verbose == 1 : print("Jogo de dado: {:.0%}".format(dice) )
        if odds_before >= dice :
            accept = 1
            accept_text = 'Sim'
        else :
            accept = 0
            accept_text = 'Não'
        accept_acc.append(accept)
        if verbose == 1 : print("Aceitou:", accept_text)

        # dado que a rede aceitou, ou não, ela aprende com isso
        output = tf.constant([accept]) 
        model.fit(input_training, output, epochs=1, verbose=0)

        # depois de ter aceitado ou não, como a opinião da rede mudou, qual a nova chance de aceitar esse mesmo caso
        odds_after = model.predict(input_training, verbose=0).squeeze()
        if verbose == 1 : print("Nova chance de aceitar esse mesmo caso: {:.0%}".format(odds_after) )
        odds_after_acc.append(odds_after.tolist())

        if verbose == 1 : print("---")

    rounds_acc.append(odds_after_acc)
print('\r')
# ------------------------------

print('Taxa de aprendizado:',l_rate)
print('Micropassos da mínima aversividade até a máxima:',steps)
print('')
print('Treinamento inicial:')

for i in range(len(output_firts)) :
    print('Para aversifidade de tarefa', input_firts[i][0],' a chance de aceitar é {:.0%}'.format(output_firts[i][0]))

only_accept_acc = [a*b for a,b in zip(accept_acc,odds_before_acc)]
only_accept_acc = [a if a!=0 else None for a in only_accept_acc]
only_not_accept_acc = [-(a-1)*b for a,b in zip(accept_acc,odds_before_acc)]
only_not_accept_acc = [a if a!=0 else None for a in only_not_accept_acc]

# plotting
f = plt.figure()
f.set_figwidth(7)
f.set_figheight(4)
plt.plot(averse_acc, odds_original_acc, "-", label='Chance de realizar a tarefa (antes do experimento)')
plt.plot(averse_acc, odds_before_acc, "-", label='Chance de realizar a tarefa (mudança durante o experimento)')
plt.plot(averse_acc, only_accept_acc, "go", label='Aceites')
plt.plot(averse_acc, only_not_accept_acc, "ro", label='Recusas')
plt.xlim(-0.1, 1.1)
plt.ylim(0, 1)
plt.legend()
plt.show()
print('')
print('')
print('')

f = plt.figure()
f.set_figwidth(7)
f.set_figheight(4)
plt.plot(averse_acc, odds_original_acc, "-", label='Chance de realizar a tarefa (antes do experimento)')
for i in range(len(rounds_acc)) :
    plt.plot(averse_acc, rounds_acc[i], "-", label='Chance de realizar a tarefa (mudança durante o experimento)', color = '.8')
plt.xlabel("Aversividade elevada em "+str(steps)+" passos, de 0 a 1")
plt.ylabel("Qual a chance de aceitar?")

plt.show()


In [ ]:
f = plt.figure()
f.set_figwidth(7)
f.set_figheight(4)
plt.plot(averse_acc, odds_original_acc, "-", label='Chance de realizar a tarefa (antes do experimento)')
for i in range(len(rounds_acc)) :
    plt.plot(averse_acc, rounds_acc[i], "-", label='Chance de realizar a tarefa (mudança durante o experimento)', color = '0', alpha=0.01)
    print(rounds_acc[i][-1])
plt.xlabel("Dificuldade elevada em "+str(steps)+" passos, de 0 a 1")
plt.ylabel("Qual a chance de aceitar?")

plt.show()


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
import matplotlib

# Treinamento inicial
## aversividade
input_firts = [[0.0],
               [0.5],
               [1.0]]

## chance de aceitar
output_firts = [[0.8],
                [0.5],
                [0.2]]

## Rede
l_rate = 0.1
model = keras.Sequential()
model.add(layers.Dense(10, activation="sigmoid"))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate), loss = tf.keras.losses.MeanSquaredError())


## Treina a rede nessas respostas
model.fit(input_firts, output_firts, epochs=100, verbose=0)

## Salva as respotas originais da rede para comparações
original_weights = model.get_weights()


steps = 4
rounds = 10
verbose = 0
rounds_acc = []
count = 0
for round in range(rounds) :
    odds_before_acc = []
    odds_original_acc = []
    odds_after_acc = []
    accept_acc = []
    averse_acc = []
    model.set_weights(original_weights)
    if verbose == 1 : print("-----------------")
    if verbose == 1 : print("### Rodada:",round)
    if verbose == 1 : print("-----------------")

    for i in range(steps+1) :
        count = count+1
        if verbose == 1 : print("Fase: ",i)
        if verbose == 1 : print("Porcentagem decorrida: ",count/(steps*rounds))
        if verbose != 1 : print("\r{:.0%}".format(count/(steps*rounds+rounds)),end="")

        # input no momento
        if verbose == 1 : print("Aversividade: ",i/steps)
        input_training =[[i/steps]]
        input_training = tf.convert_to_tensor(input_training)
        averse_acc.append(i/steps)

        # resposta original que a rede daria, para fins de comparação
        curr_weights = model.get_weights()
        model.set_weights(original_weights)
        odds_original = model.predict(input_training, verbose=0).squeeze()
        if verbose == 1 : print("Originalmente a chance era: {:.0%}".format(odds_original) )
    
        # volta a rede para o estado atual
        odds_original_acc.append(odds_original.tolist())
        model.set_weights(curr_weights)

        # resposta que a rede dá nesse momento, antes de aceitar ou não a solicitação
        odds_before = model.predict(input_training, verbose=0).squeeze()
        if verbose == 1 : print("Chance de aceitar: {:.0%}".format(odds_before) )
        odds_before_acc.append(odds_before.tolist())

        # sendo a resposta atual uma probabilidade, a rede vai aceitar ou não
        dice = random.random()
        if verbose == 1 : print("Jogo de dado: {:.0%}".format(dice) )
        if odds_before >= dice :
            accept = 1
            accept_text = 'Sim'
        else :
            accept = 0
            accept_text = 'Não'
        accept_acc.append(accept)
        if verbose == 1 : print("Aceitou:", accept_text)

        # dado que a rede aceitou, ou não, ela aprende com isso
        output = tf.constant([accept]) 
        model.fit(input_training, output, epochs=1, verbose=0)

        # depois de ter aceitado ou não, como a opinião da rede mudou, qual a nova chance de aceitar esse mesmo caso
        odds_after = model.predict(input_training, verbose=0).squeeze()
        if verbose == 1 : print("Nova chance de aceitar esse mesmo caso: {:.0%}".format(odds_after) )
        odds_after_acc.append(odds_after.tolist())

        if verbose == 1 : print("---")

    rounds_acc.append(odds_after_acc)
print('\r')
# ------------------------------

print('Taxa de aprendizado:',l_rate)
print('Micropassos da mínima aversividade até a máxima:',steps)
print('')
print('Treinamento inicial:')

for i in range(len(output_firts)) :
    print('Para aversifidade de tarefa', input_firts[i][0],' a chance de aceitar é {:.0%}'.format(output_firts[i][0]))

only_accept_acc = [a*b for a,b in zip(accept_acc,odds_before_acc)]
only_accept_acc = [a if a!=0 else None for a in only_accept_acc]
only_not_accept_acc = [-(a-1)*b for a,b in zip(accept_acc,odds_before_acc)]
only_not_accept_acc = [a if a!=0 else None for a in only_not_accept_acc]

# plotting
f = plt.figure()
f.set_figwidth(30)
f.set_figheight(10)
plt.plot(averse_acc, odds_original_acc, "-", label='Chance de realizar a tarefa (antes do experimento)')
plt.plot(averse_acc, odds_before_acc, "-", label='Chance de realizar a tarefa (mudança durante o experimento)')
plt.plot(averse_acc, only_accept_acc, "go", label='Aceites')
plt.plot(averse_acc, only_not_accept_acc, "ro", label='Recusas')
plt.xlim(-0.1, 1.1)
plt.ylim(0, 1)
plt.legend()
plt.show()
print('')
print('')
print('')

f = plt.figure()
f.set_figwidth(30)
f.set_figheight(10)
plt.plot(averse_acc, odds_original_acc, "-", label='Chance de realizar a tarefa (antes do experimento)')
for i in range(len(rounds_acc)) :
    plt.plot(averse_acc, rounds_acc[i], "-", label='Chance de realizar a tarefa (mudança durante o experimento)', color = '.8')
plt.xlabel("Aversividade elevada em "+str(steps)+" passos, de 0 a 1")
plt.ylabel("Qual a chance de aceitar?")

plt.show()


# Caso 2: Conformação com maioria.

Objetivo do caso: Observar como redes livres para se comunicarem chegam a um consenso, sendo uma rede com opinião divergente

Treinamento inicial: N redes são treinada com determinadas posições sobre uma entrada.

Predição: uma RN indica sua opinião a respeito de uma entrada.

Ação: todas as redes ouvems essa opinião.

Aprendizado: todas as redes aprendem com essa opinião

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

print('Iniciando.', end = '')

participants = 6

original_model = []
n = participants-1
threshold = 0.8
## tema geral
theme = [[.5]]

# Treinamento inicial sujeito

## opinião da rede sujeito
subj_op = 0.1
opinion = [[subj_op]]

## Rede sujeito
# l_rate_subj = 0.5
# model = keras.Sequential()
# model.add(layers.Dense(10, activation="sigmoid"))
# model.add(layers.Dense(1, activation="sigmoid"))
# model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=l_rate_subj), loss = tf.keras.losses.MeanSquaredError())


l_rate_subj = 0.1
model = keras.Sequential()
model.add(layers.Dense(10, activation="sigmoid"))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate_subj), loss = tf.keras.losses.MeanSquaredError())

## Treina a rede nessas respostas
model.fit(theme, opinion, epochs=1000, verbose=0)

## Salva as respotas originais da rede para comparações
original_model.append(model)

# Treinamento inicial outras redes

for i in range(1,participants):

    ## opinião da rede sujeito
    opinion = [[random.random()*(1-threshold)+threshold]]

    ## Rede
    # l_rate = 0.5
    # model = keras.Sequential()
    # model.add(layers.Dense(10, activation="sigmoid"))
    # model.add(layers.Dense(10, activation="sigmoid"))
    # model.add(layers.Dense(1, activation="sigmoid"))
    # model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=l_rate), loss = tf.keras.losses.MeanSquaredError())
    l_rate = 0.1
    model = keras.Sequential()
    model.add(layers.Dense(10, activation="sigmoid"))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate), loss = tf.keras.losses.MeanSquaredError())

    ## Treina a rede nessas respostas
    model.fit(theme, opinion, epochs=100, verbose=0)

    ## Salva as respotas originais da rede para comparações
    original_model.append(model)


# copia os pesos que são as opiniões de todas as redes
curr_model = original_model.copy()
print('\r', end = '')



rounds = 30
rounds = rounds+1
acc_opinions = np.zeros((rounds,participants))
count = 0
verbose = 0

# opniões iniciais
for i in range(participants) :
    model = curr_model[i]
    opinion = model.predict(theme, verbose=0).tolist()
    acc_opinions[0][i] = opinion[0][0]
    count = count + 1
    if verbose != 1 : print('\r{:.0%}'.format(count/(rounds*participants+participants)),end='')

for round in range(1,rounds) :
    for i in range(participants) :
        if verbose == 1 : print('---')
        if verbose == 1 : print('Percentual de conclusão: {:.0%}'.format(count/(rounds*participants)))
        if verbose != 1 : print('\r{:.0%}'.format(count/(rounds*participants+participants)),end='')

        model = curr_model[i]
        opinion = model.predict(theme, verbose=0).tolist()
        if verbose == 1 : print("O participante " + str(i) + " dá sua opinião: {:.2}".format(opinion[0][0]) )

        acc_opinions[round][i] = opinion[0][0]
        count = count + 1
        for j in range(participants) :
            # todas as rede aprendem com a opinião da rodada, exceto ela mesma que está dando a opinião
            if i != j :
                model = curr_model[j]
                curr_opinion = model.predict(theme, verbose=0).squeeze()
                if verbose == 1 : print("A opinião do participante " + str(j) + " era: {:.2}".format(curr_opinion) )
                model.fit(theme, opinion, epochs=1, verbose=0)
                curr_opinion = model.predict(theme, verbose=0).squeeze()
                if verbose == 1 : print("E passou a ser: {:.2}".format(curr_opinion) )

                acc_opinions[round][j] = curr_opinion
                curr_model[j] = model
        if verbose == 1 : print("")

# ------------------------------
print('\r--------------------------------------------------------------------')
print('Número de participantes:',participants)
print('Taxa de aprendizado do sujeito:',l_rate_subj)
print('Taxa de aprendizado dos demais:',l_rate)
print('Rodadas de conversa onde cada um fala e todos ouvem:',rounds-1)
print('Opinião do sujeito:', subj_op)
print('Opinião mínima dos outros:', threshold)
print('--------------------------------------------------------------------')

f = plt.figure()
f.set_figwidth(30)
f.set_figheight(10)

xs = [x for x in range(len(acc_opinions.T[0]))]

plt.plot(xs, acc_opinions, color = '.8')
plt.plot(xs, acc_opinions.T[0], "r-")
plt.show()


# Caso 3: Poder da minoria.

Objetivo do caso: Observar como redes livres para se comunicarem chegam a um consenso, sendo uma rede com opinião divergente e com learning rate muito baixo

Treinamento inicial: N redes são treinada com determinadas posições sobre uma entrada.

Predição: uma RN indica sua opinião a respeito de uma entrada.

Ação: todas as redes ouvems essa opinião.

Aprendizado: todas as redes aprendem com essa opinião

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

print('Iniciando.', end = '')

participants = 6

original_model = []
n = participants-1
threshold = 0.8
## tema geral
theme = [[.5]]

# Treinamento inicial sujeito

## opinião da rede sujeito
subj_op = 0.1
opinion = [[subj_op]]

## Rede sujeito
# l_rate_subj = 0.5
l_rate_subj = 0.001
model = keras.Sequential()
model.add(layers.Dense(10, activation="sigmoid"))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate_subj), loss = tf.keras.losses.MeanSquaredError())

## Treina a rede nessas respostas
model.fit(theme, opinion, epochs=1000, verbose=0)

## Salva as respotas originais da rede para comparações
original_model.append(model)

# Treinamento inicial outras redes

for i in range(1,participants):

    ## opinião da rede sujeito
    opinion = [[random.random()*(1-threshold)+threshold]]

    ## Rede
    # l_rate = 0.5
    l_rate = 0.1
    model = keras.Sequential()
    model.add(layers.Dense(10, activation="sigmoid"))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate), loss = tf.keras.losses.MeanSquaredError())

    ## Treina a rede nessas respostas
    model.fit(theme, opinion, epochs=100, verbose=0)

    ## Salva as respotas originais da rede para comparações
    original_model.append(model)


# copia os pesos que são as opiniões de todas as redes
curr_model = original_model.copy()
print('\r', end = '')



rounds = 30
rounds = rounds+1
acc_opinions = np.zeros((rounds,participants))
count = 0
verbose = 0

# opniões iniciais
for i in range(participants) :
    model = curr_model[i]
    opinion = model.predict(theme, verbose=0).tolist()
    acc_opinions[0][i] = opinion[0][0]
    count = count + 1
    if verbose != 1 : print('\r{:.0%}'.format(count/(rounds*participants+participants)),end='')

for round in range(1,rounds) :
    for i in range(participants) :
        if verbose == 1 : print('---')
        if verbose == 1 : print('Percentual de conclusão: {:.0%}'.format(count/(rounds*participants)))
        if verbose != 1 : print('\r{:.0%}'.format(count/(rounds*participants+participants)),end='')

        model = curr_model[i]
        opinion = model.predict(theme, verbose=0).tolist()
        if verbose == 1 : print("O participante " + str(i) + " dá sua opinião: {:.2}".format(opinion[0][0]) )

        acc_opinions[round][i] = opinion[0][0]
        count = count + 1
        for j in range(participants) :
            # todas as rede aprendem com a opinião da rodada, exceto ela mesma que está dando a opinião
            if i != j :
                model = curr_model[j]
                curr_opinion = model.predict(theme, verbose=0).squeeze()
                if verbose == 1 : print("A opinião do participante " + str(j) + " era: {:.2}".format(curr_opinion) )
                model.fit(theme, opinion, epochs=1, verbose=0)
                curr_opinion = model.predict(theme, verbose=0).squeeze()
                if verbose == 1 : print("E passou a ser: {:.2}".format(curr_opinion) )

                acc_opinions[round][j] = curr_opinion
                curr_model[j] = model
        if verbose == 1 : print("")

# ------------------------------
print('\r--------------------------------------------------------------------')
print('Número de participantes:',participants)
print('Taxa de aprendizado do sujeito:',l_rate_subj)
print('Taxa de aprendizado dos demais:',l_rate)
print('Rodadas de conversa onde cada um fala e todos ouvem:',rounds-1)
print('Opinião do sujeito:', subj_op)
print('Opinião mínima dos outros:', threshold)
print('--------------------------------------------------------------------')

f = plt.figure()
f.set_figwidth(30)
f.set_figheight(10)

xs = [x for x in range(len(acc_opinions.T[0]))]

plt.plot(xs, acc_opinions, color = '.8')
plt.plot(xs, acc_opinions.T[0], "r-")
plt.show()


# Caso 4: Polarização.

Objetivo do caso: Observar como redes livres para se comunicarem chegam a um consenso, mas caso uma divergêngia em certo patamar gerar repulsão à ideia, uma polarização acontece

Treinamento inicial: N redes são treinada com determinadas posições sobre uma entrada.

Predição: uma RN indica sua opinião a respeito de uma entrada.

Ação: todas as redes ouvems essa opinião.

Aprendizado: todas as redes aprendem com essa opinião se a divergência não for muito grande, se a divergência for grande, ela desloca sua opinião para uma posição contrária com uma certa intensidade

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

print('Iniciando.')

participants = 6
original_model = []
n = participants

## tema geral
theme = [[.5]]

# Treinamento inicial redes

for i in range(participants):
    print('Treinamento inicial sujeito: ', i)
    ## opinião da rede sujeito
    opinion = [[random.random()]]
    opinion = [[1/(participants-1)*i]]
    print('Opinião:',opinion)
    print()
    ## Rede
    l_rate = 0.1
    model = keras.Sequential()
    model.add(layers.Dense(10, activation="sigmoid"))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=l_rate), loss = tf.keras.losses.MeanSquaredError())

    ## Treina a rede nessas respostas
    model.fit(theme, opinion, epochs=100, verbose=0)

    ## Salva as respotas originais da rede para comparações
    original_model.append(model)


# copia os pesos que são as opiniões de todas as redes
curr_model = original_model.copy()
print('\r', end = '')

rounds = 30
rounds = rounds+1
acc_opinions = np.zeros((rounds,participants))
count = 0
verbose = 0

diverge = 1
intensity = 0.7

# opniões iniciais
for i in range(participants) :
    model = curr_model[i]
    opinion = model.predict(theme, verbose=0).tolist()
    acc_opinions[0][i] = opinion[0][0]
    count = count + 1
    if verbose != 1 : print('\r{:.0%}'.format(count/(rounds*participants+participants)),end='')

for round in range(1,rounds) :
    for i in range(participants) :
        model = curr_model[i]
        opinion = model.predict(theme, verbose=0).tolist()

        acc_opinions[round][i] = opinion[0][0]
        count = count + 1
        for j in range(participants) :
            # todas as rede aprendem com a opinião da rodada, exceto ela mesma que está dando a opinião
            if i != j :
                # opinião de quem fala
                opinion = acc_opinions[round][i]

                model = curr_model[j]
                curr_listen = model.predict(theme, verbose=0).squeeze()

                if abs(opinion-curr_listen) <= diverge:
                    opinion = opinion
                else :
                    opinion = curr_listen + (curr_listen-opinion)*intensity
                    if opinion > 1 : 
                        opinion = 1
                    elif opinion < 0 : 
                        opinion = 0
                opinion = [[opinion]]

                model.fit(theme, opinion, epochs=1, verbose=0)
                curr_opinion = model.predict(theme, verbose=0).squeeze()

                acc_opinions[round][j] = curr_opinion
                curr_model[j] = model
        if verbose == 1 : print("")

# ------------------------------
print('\r--------------------------------------------------------------------')
print('Número de participantes:',participants)
print('Taxa dos sujeito:',l_rate)
print('Rodadas de conversa onde cada um fala e todos ouvem:',rounds-1)
# print('Opinião mínima dos outros:', threshold)
print('--------------------------------------------------------------------')

f = plt.figure()
f.set_figwidth(30)
f.set_figheight(10)

xs = [x for x in range(len(acc_opinions.T[0]))]

plt.plot(xs, acc_opinions, color = '.8')
plt.show()

print()
print()
print()
print()
print()
print()
print()


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

print('Iniciando.')

participants = 6
original_model = []
n = participants

## tema geral
theme = [[.5]]

# Treinamento inicial outras redes

for i in range(participants):
    print('Treinamento inicial sujeito: ', i)
    ## opinião da rede sujeito
    opinion = [[random.random()]]
    opinion = [[1/(participants-1)*i]]
    print('Opinião:',opinion)
    print()
    ## Rede
    l_rate = 0.1
    model = keras.Sequential()
    model.add(layers.Dense(10, activation="sigmoid"))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=l_rate), loss = tf.keras.losses.MeanSquaredError())

    ## Treina a rede nessas respostas
    model.fit(theme, opinion, epochs=100, verbose=0)

    ## Salva as respotas originais da rede para comparações
    original_model.append(model)


# copia os pesos que são as opiniões de todas as redes
curr_model = original_model.copy()
print('\r', end = '')

rounds = 30
rounds = rounds+1
acc_opinions = np.zeros((rounds,participants))
count = 0
verbose = 0

diverge = 0.25
intensity = 0.7

# opniões iniciais
for i in range(participants) :
    model = curr_model[i]
    opinion = model.predict(theme, verbose=0).tolist()
    acc_opinions[0][i] = opinion[0][0]
    count = count + 1
    if verbose != 1 : print('\r{:.0%}'.format(count/(rounds*participants+participants)),end='')

for round in range(1,rounds) :
    for i in range(participants) :
        if verbose == 1 : print('---')
        if verbose == 1 : print('Percentual de conclusão: {:.0%}'.format(count/(rounds*participants)))
        if verbose != 1 : print('\r{:.0%}'.format(count/(rounds*participants+participants)),end='')
        model = curr_model[i]
        opinion = model.predict(theme, verbose=0).tolist()
        if verbose == 1 : print("O participante " + str(i) + " dá sua opinião: {:.2}".format(opinion[0][0]) )

        acc_opinions[round][i] = opinion[0][0]
        count = count + 1
        for j in range(participants) :
            # todas as rede aprendem com a opinião da rodada, exceto ela mesma que está dando a opinião
            if i != j :
                # opinião de quem fala
                opinion = acc_opinions[round][i]

                model = curr_model[j]
                curr_listen = model.predict(theme, verbose=0).squeeze()

                if abs(opinion-curr_listen) <= diverge:
                    opinion = opinion
                else :
                    opinion = curr_listen + (curr_listen-opinion)*intensity
                    if opinion > 1 : 
                        opinion = 1
                    elif opinion < 0 : 
                        opinion = 0
                opinion = [[opinion]]

                model.fit(theme, opinion, epochs=1, verbose=0)
                curr_opinion = model.predict(theme, verbose=0).squeeze()

                acc_opinions[round][j] = curr_opinion
                curr_model[j] = model
        if verbose == 1 : print("")

# ------------------------------
print('\r--------------------------------------------------------------------')
print('Número de participantes:',participants)
print('Taxa dos sujeito:',l_rate)
print('Rodadas de conversa onde cada um fala e todos ouvem:',rounds-1)
# print('Opinião mínima dos outros:', threshold)
print('--------------------------------------------------------------------')

f = plt.figure()
f.set_figwidth(30)
f.set_figheight(10)

xs = [x for x in range(len(acc_opinions.T[0]))]

plt.plot(xs, acc_opinions, color = '.8')
plt.show()

print()
print()
print()
print()
print()
print()
print()


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

print('Iniciando.')

participants = 20
original_model = []
n = participants

## tema geral
theme = [[.5]]

# Treinamento inicial outras redes

for i in range(participants):
    print('Treinamento inicial sujeito: ', i)
    ## opinião da rede sujeito
    opinion = [[random.random()]]
    # opinion = [[1/(participants-1)*i]]
    print('Opinião:',opinion)
    print()
    ## Rede
    l_rate = 0.1
    model = keras.Sequential()
    model.add(layers.Dense(10, activation="sigmoid"))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=l_rate), loss = tf.keras.losses.MeanSquaredError())

    ## Treina a rede nessas respostas
    model.fit(theme, opinion, epochs=100, verbose=0)

    ## Salva as respotas originais da rede para comparações
    original_model.append(model)


# copia os pesos que são as opiniões de todas as redes
curr_model = original_model.copy()
print('\r', end = '')

rounds = 30
rounds = rounds+1
acc_opinions = np.zeros((rounds,participants))
count = 0
verbose = 0

diverge = 0.25
intensity = 0.7

# opniões iniciais
for i in range(participants) :
    model = curr_model[i]
    opinion = model.predict(theme, verbose=0).tolist()
    acc_opinions[0][i] = opinion[0][0]
    count = count + 1
    if verbose != 1 : print('\r{:.0%}'.format(count/(rounds*participants+participants)),end='')

for round in range(1,rounds) :
    for i in range(participants) :
        if verbose == 1 : print('---')
        if verbose == 1 : print('Percentual de conclusão: {:.0%}'.format(count/(rounds*participants)))
        if verbose != 1 : print('\r{:.0%}'.format(count/(rounds*participants+participants)),end='')
        model = curr_model[i]
        opinion = model.predict(theme, verbose=0).tolist()
        if verbose == 1 : print("O participante " + str(i) + " dá sua opinião: {:.2}".format(opinion[0][0]) )

        acc_opinions[round][i] = opinion[0][0]
        count = count + 1
        for j in range(participants) :
            # todas as rede aprendem com a opinião da rodada, exceto ela mesma que está dando a opinião
            if i != j :
                # opinião de quem fala
                opinion = acc_opinions[round][i]

                model = curr_model[j]
                curr_listen = model.predict(theme, verbose=0).squeeze()

                if abs(opinion-curr_listen) <= diverge:
                    opinion = opinion
                else :
                    opinion = curr_listen + (curr_listen-opinion)*intensity
                    if opinion > 1 : 
                        opinion = 1
                    elif opinion < 0 : 
                        opinion = 0
                opinion = [[opinion]]

                model.fit(theme, opinion, epochs=1, verbose=0)
                curr_opinion = model.predict(theme, verbose=0).squeeze()

                acc_opinions[round][j] = curr_opinion
                curr_model[j] = model
        if verbose == 1 : print("")

# ------------------------------
print('\r--------------------------------------------------------------------')
print('Número de participantes:',participants)
print('Taxa dos sujeito:',l_rate)
print('Rodadas de conversa onde cada um fala e todos ouvem:',rounds-1)
# print('Opinião mínima dos outros:', threshold)
print('--------------------------------------------------------------------')

f = plt.figure()
f.set_figwidth(30)
f.set_figheight(10)

xs = [x for x in range(len(acc_opinions.T[0]))]

plt.plot(xs, acc_opinions, color = '.8')
plt.show()

print()
print()
print()
print()
print()
print()
print()
